# Time classification

In [301]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, roc_auc_score, mean_squared_error
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import datetime
rnd = 12

In [326]:
#df = pd.read_csv('df_fin.csv')
df = pd.read_excel('df_fin1.xlsx')

In [327]:
# I will use only most useful created features from previous model
# and so only email related features will be included
# Complete datatset could be created using same techniques as below combined with 
# data prep pipeline from Binary regression

In [328]:
#model_vars = ['customer_id','date','hour','weekday',
#              'prch_freq','prch_last','prch_mean_diff',
#              'cart_freq','cart_last','cart_mean_diff',
#              'view_freq','view_last','view_mean_diff',
#              'sess_freq','sess_last','sess_mean_diff',
#              'email_freq','email_last','email_mean_diff']

model_vars = ['customer_id','date','hour','weekday',
              'email_freq','email_last','email_mean_diff']
model_df = df[model_vars]
Y = df.response
model_df = model_df[Y==1] 

In [329]:
[model_df.date.min(),model_df.date.max()]

[Timestamp('2018-03-17 08:03:12.581000'),
 Timestamp('2018-05-15 15:01:16.181000')]

In [330]:
# Find range of possible values and sample from it
rng = int(abs(model_df.date.min() - model_df.date.max()).total_seconds())
mn = model_df.date.min()

In [331]:
# if any event occured inbetween it is involuntarilky excluded

In [332]:
df = pd.DataFrame(columns=model_df.columns)

for x in range(200):
    # transform observation to timestamp
    i = int(np.random.uniform(0,rng))
    stamp = mn + datetime.timedelta(seconds=i)

    # create relevant subset of positive reponses
    # ie y=0 & is in the future for each customer
    temp_df = model_df[model_df.date > stamp]
    temp_df.sort_values(by='date',inplace=True)
    temp_df.drop_duplicates(subset='customer_id',inplace=True)

    diff = (temp_df.date - stamp).dt.seconds.fillna(0).div(60)
    temp_df.hour = stamp.hour
    temp_df.weekday = stamp.dayofweek + 1
    temp_df.email_last = model_df.email_last - diff
    # response in hours
    temp_df['cresponse'] = diff.div(60)

    df = df.append(temp_df[temp_df.email_last>=0])
df = pd.DataFrame(df)

C:\Users\chladek\AppData\Local\Continuum\anaconda3-64\envs\tf\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\chladek\AppData\Local\Continuum\anaconda3-64\envs\tf\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\chladek\AppData\Local\Continuum\anaconda3-64\envs\tf\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [333]:
df

,cresponse,customer_id,date,email_freq,email_last,email_mean_diff,hour,weekday
12983,12.925000,319c2b1241efdd2f06620b24,2018-05-10 06:42:56.777,1,403.616667,0.000000,17,3
42645,12.925000,a5b9aabd4b0908fc5b23e057,2018-05-10 06:42:57.365,28,614.416667,683.225926,17,3
154,12.925278,00b23eb6c75112d8c72ec42e,2018-05-10 06:42:57.614,8,433.300000,762.285714,17,3
43836,12.925278,aaa83387048738d7b37b6531,2018-05-10 06:42:58.258,33,638.800000,671.650000,17,3
30119,12.925556,705f9509777b0348a1097b8c,2018-05-10 06:42:58.715,35,615.633334,761.425000,17,3
13071,12.925556,31c1adee13e1acb6118e94c3,2018-05-10 06:42:58.822,13,373.066667,854.973611,17,3
33863,12.925556,8123bf5ad61841fef63f135a,2018-05-10 06:42:59.528,9,588.266667,502.912500,17,3
23412,12.925833,56bbba9e9771e7c37a8c6a52,2018-05-10 06:42:59.596,31,507.450000,767.089444,17,3
26953,12.925833,649ccb3732332630ea887127,2018-05-10 06:42:59.809,34,480.850000,646.504545,17,3
12770,12.926111,30723b2d7b6bf48d82e9d5fc,2018-05-10 06:43:00.584,33,642.600000,626.373958,17,3


In [340]:
#for col in ['weekday']:
#    dummies = pd.get_dummies(df[col], prefix=col)
#    to_del = pd.get_dummies(df[col], prefix=col).columns[0]
#    dummies.drop(labels=to_del, axis=1, inplace=True)
#    df = pd.concat([df,dummies],axis=1)
#    df.drop([col],axis=1, inplace=True)

X = df.drop(['cresponse','customer_id','date'],1)
Y = df.cresponse.astype(int)

In [342]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=rnd)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
y_test_transformed = np.hstack((1 - y_test.reshape(y_test.size,1),
                                y_test.reshape(y_test.size,1)))

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(65000, 5) (65000,)
(7223, 5) (7223,)


In [343]:
Y.value_counts()

7     5075
6     4868
3     4514
8     4424
15    4296
11    4141
5     3975
2     3929
13    3788
16    3305
10    3218
4     3022
12    3013
9     2511
14    2502
18    2454
17    2260
0     2243
1     2158
19    2048
21    1628
22    1226
20    1173
23     452
Name: cresponse, dtype: int64

# Classes

In [344]:
# quick RF model baseline
clf_rf = RandomForestClassifier(n_estimators=300, random_state=rnd,verbose=1)
clf_rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   58.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=12, verbose=1, warm_start=False)

# Regression

In [299]:
# quick RF model baseline
rgr_rf = RandomForestRegressor(n_estimators=300, random_state=rnd,verbose=1)
rgr_rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  1.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
           oob_score=False, random_state=12, verbose=1, warm_start=False)

In [325]:
Y.min()

0.0005555555555555556

In [320]:
print('Test Results')
print("MSE: ",mean_squared_error(y_test,rgr_rf.predict(X_test)))

Test Results
MSE:  3.256871210934655


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.1s finished


In [300]:
var_imp_rgr_rf = pd.DataFrame({'vars': X_train.columns,'imp':rgr_rf.feature_importances_})
var_imp_rgr_rf.sort_values(by='imp',ascending=False)

,vars,imp
3,hour,0.810519
1,email_last,0.141308
2,email_mean_diff,0.030434
0,email_freq,0.010584
4,weekday,0.007155
